# Cognitive Neuroscience: Group Project

## Final Group Project Code Instructions

Marijn van Wingerden, Department of Cognitive Science and Artificial Intelligence – Tilburg University Academic Year 2020-2021

In this Jupyter Notebook, you will find the programmatic instructions to complete the Group Project. 

You will analyse a subset of the trials and conditions from the RITA dataset, as introduced by dr. Roncaglia in the second week of class. Your time-resolved analysis, using the FFT and baseline averaging, will be performed across ALL subjects in a group (Monolinguals, Early Learners, Late Learners) in the dataset, and look for oscillatory activity in relation to the onset of the **critical item** in each sentence (the auxilary verb).

You will inspect activity in the following frequency bands:
- Delta (1-4 Hz)
- Theta (4-8 Hz)
- Alpha (8-12 Hz)
- Beta (15-25 Hz)
- low Gamma (30-60 Hz)
- high Gamma (60-100 Hz)
(different cutoffs points can be found in the literature, we are sticking with these for convenience)

All relevant methods have been covered in Worksheets 1-7, with the exception of loading multiple datafiles. Whenever a new method/function is introduced here, it will come with an example.


## Datafiles - assignment per group

Each group will analyze one set of filler sentences (NA: Non-Ambiguous sentences) and a set of experimental sentences (AM: Ambiguous sentences). The conditions in the dataset are split over groups in the following way:

- SF: subject-first
- OF: object-first
- IR: irregular rhythm
- RR: regular rhythm

- Monolinguals: RM
    - Groups 01 + 02: SFIR triggers XX1 and XX5
    - Groups 03 + 04: SFRR triggers XX2 and XX6
    - Groups 05 + 06: OFIR triggers XX3 and XX7
    - Groups 07 + 08: OFRR triggers XX4 and XX8
- Early Learners: RB
    - Groups 09 + 10: SFIR triggers XX1 and XX5
    - Groups 11 + 12: SFRR triggers XX2 and XX6
    - Groups 13 + 14: OFIR triggers XX3 and XX7
    - Groups 15 + 16: OFRR triggers XX4 and XX8
- Late Learners: RL
    - Groups 17 + 18: SFIR triggers XX1 and XX5
    - Groups 19 + 20: SFRR triggers XX2 and XX6
    - Groups 21 + 22: OFIR triggers XX3 and XX7
    - Groups 23 + 24: OFRR triggers XX4 and XX8

- Odd groups:
analyse the odd trials in the dataset
- Even groups:
analyse the even trials in the dataset

You can download the datafiles from: https://surfdrive.surf.nl/files/index.php/s/JcA9speED020q4p


## Handing in of your code

You can adapt this script template and hand it in as the code component of your Group Assignment Report.

Whenever you are asked to make a plot, it should be completed with a meaningful plot title, xlabel and ylabel texts. Figures are started with a Matplotlib figure handle: "fig_Q2A, ax = plt.subplots;". This indicates that a link (called handle) to your figure will be saved in the variable, so we can easily check it when checking your scripts. Whenever a naming convention for a variable is given, use it, because it will allow semi-automatic grading of your project script.

### Intermediate hand-in

You will be able to hand in a script/notebook with your solutions to Q1-3 after the midterms, and the correct solution will be discussed in class to give you some feedback on how the Group Project is evaluated. Groups that do not hand in their solutions to Q1-3 will receive half of the total points available for these questions by default. 

## Group members:

Please list the contributors and their U-numbers here in comments:

- u838557 | Vivienne Kraeter   
- u133313 | Frederic Straub
- u947993 | Jorge Horan-Barnes 
- u848541 | Tom de Beer
- u551835 | Hamse Elmi

## Setting up: list your modules to import
For loading/saving puroposes, we will make use of the **os** package.
An example worksheet with instructions on how to use the os package will be provided

In [3]:
%matplotlib notebook

import scipy.io as spio
import scipy.fft as fft
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

## Data loading
We will need to load the datafiles from all participants and add them all together so that we end up with a matrix that has nChannels x nTime x nParticipants (instead of trials). You can make your work easier by organising the datafiles in such a way that you put the control.npy files in their own subdirectory, and the experimental.npy files as well. 

In order to load the files, we can use the os package.

Adapt the following so that it works on your machine:

In [1]:
path_control = "C:\\Users\\hamse\\OneDrive\\Documenten\\group_17\\Control"
path_experimental = "C:\\Users\\hamse\\OneDrive\\Documenten\\group_17\\Experiment"
control_files = os.listdir(path_control)
experimental_files = os.listdir(path_experimental)

# check that the length of your files list matches the provided datafiles, and contains only .npy datafiles

def checkLengthAndType(pathI):
    count = 0
    for path in os.listdir(pathI):
        if os.path.isfile(os.path.join(pathI, path)) and path.endswith('.npy'):
            count += 1
    if(count==26):
        return True
    return False
checkLengthAndType(path_control)
checkLengthAndType(path_experimental)

True

## Combining data and matrix pre-allocation
next, you will need to load these files one by one and extract the data for this participant. 
The data in the NumPy arrays are stored as Trials x Channels x Time. To aggregate across participants, you will thus need to add a 4th dimension to store the data.

To be able to adequately pre-allocate the data from the different subjects, we will load one trial subject manually to have a look at the shape/dimensionality of the data:

In [9]:
EEG = np.load(os.path.join(path_control,control_files[0]))
              
# control_files is a list of strings, so indexing its first element returns a string
# in this case, we are loading the first entry of control_files, i.e. participant 1

 

# verify that the number of trials equals 22,
if(EEG.shape[0]==22):
    print("The amount of trials is equal to 22")
# verify that the number of channels equals 64 or 65 
if(EEG.shape[1]==64 or EEG.shape(1)==65):
    print("The amount of channels is equal to either 64 or 65")
# and verify that there are 751 samples per trace
if(EEG.shape[2]==751):
    print("The amount of samples/trace is equal to 751")



The amount of trials is equal to 22
The amount of channels is equal to either 64 or 65
The amount of samples/trace is equal to 751


## Q1 - setting up the data structure and loading data from all participants

The EEG data is currently stored as a 3-dimensional NumPy array. But to run our time-frequency analysis, we need some more information like the sampling rate and the time axis that corresponds to the stimulus-locked analysis window. In order to set up (=pre-allocate) a matrix that will hold all traces for all participants, we need to know the sizes of the dimensions of this 4-dimensional matrix, and fill up this matrix by looping over participants:

In [19]:
# There are 64 or 65 channels in the dataset. Only channels up to channel 59 are EEG channels
# the remaining channels are EMG and EOG channels that we will ignore in this analysis
# subset your EEG array so that only the EEG channels remain

EEG = []
# the channels above 59 are removed from the control dataset
for x in range(0,len(control_files)):
    EEG = np.load(os.path.join(path_control,control_files[x]))
    EEG = EEG[:,:59,:]
# the channels above 59 are removed from the experimental dataset
for x in range(0,len(experimental_files)):
    EEG = np.load(os.path.join(path_experimental,experimental_files[x]))
    EEG = EEG[:,:59,:]

# Define nTrials, nChan, nSamples and nPart. Then, pre-allocate a matrix
# named comb_data, filled with zeros and with size nTrials, nChans x nSamples x nParticipants
nTrials = 22
nChans = 59 # number of channels
nSamples = 751
nParts = 26 # number of participants
comb_data_control = np.zeros(shape=(nTrials,nChans,nSamples,nParts)) # 4 dimensional control-dataset array
comb_data_experimental = np.zeros(shape=(nTrials,nChans,nSamples,nParts)) # 4 dimensional experimental-dataset array

# next, we need to loop over all participant datafiles and add them to the appropriate slice in your 4-D array
# For this, you need to use specific array indexing to indicate where in comb_data each participant's data
# needs to go. You can and should reuse the data-reading code above.

# loop over participants, and wihtin each iteration of the loop, load the
# next datafile and fill comb_data with the EEG traces (nTrials x nChans x nSamples)
# caution - multiple lines of code might be needed

#looping over control files to fill comb_data_control
for participant in range(len(control_files)):
    for i in range(EEG.shape[0]):
        for j in range(EEG.shape[1]):
            for x in range(EEG.shape[2]):
                comb_data_control[i,j,x,participant] = EEG[i,j,x]

#looping over experimental files to fill comb_data_experimental
for participant in range(len(experimental_files)):
    for i in range(EEG.shape[0]):
        for j in range(EEG.shape[1]):
            for x in range(EEG.shape[2]):
                comb_data_experimental[i,j,x,participant] = EEG[i,j,x]

KeyboardInterrupt: 

## Q2 - explore the data

Let's explore this newly combined dataset a little bit. This sections fo EEG traces in your dataset have been taken with a [-0.5s, 1s] window aroun the relevant event. What's more, each trace has been averaged to its baseline period, so that the mean amplitude should be 0 (with some rounding error). 

To verify, first, determine the mean for the time period of -0.5 to 0 seconds. Given that the srate = 500 Hz, the baseline period corresponds to the first 250 samples
- subset your combined data to only the first 250 samples
- select a random participant and sunset the data further to only this participant
- select a random EEG channel and subset the data further to only this channel

This should leave you with a nTrials x 250 (samples) matrix. Create a similar evoked matrix with the remainder of the samples. With these matrices:
- plot the traces for all trials in the **baseline** matrix (use transpose if necessary). 
- calculate the mean for each trace:
    - once for the baseline period
    - once for the remainder of the trial
- plot these values (N= nTrials, check) in a histogram, each in their own subplot

Refer to Worksheet 1 for example uses of **np.mean**. np.std works in a similar way

In [ ]:
baseline_mean = ...
evoked_mean = ...

# now plot these three histograms using the subplot given
fig_Q2A, ax = plt.subplots(figsize=(8,3), nrows=1, ncols=3) # 1x3 graph

# plot the traces in ax[0]

##
## your code here
##

# plot the baseline mean in ax[1] with this code
ax[1].hist(baseline_mean)
plt.axes(ax[1])
plt.title('Baseline means')
ax[1].set_xlabel('Mean voltage')
ax[1].set_ylabel('count')

# plot the evoked mean in ax[2] with this code
ax[2].hist(evoked_mean)
plt.axes(ax[2])
plt.title('Evoked means')
ax[2].set_xlabel('Mean voltage')
ax[2].set_ylabel('count')

Looking at theses histograms, you should see that the distribution of evoked mean values can vary: while EEG recordings are referenced to a common ground, and usualy then re-referenced to the global average, we are dealing here with specific cutouts of EEG traces around specific events in the dataset. The data has been normalised to the baseline window for each trial, but the mean of the evoked part is not controlled. In addition, there might be differences in the size of the amplitudes between channels and between participants due to differences in conductivity.

Illustrate that this global averaging does not guarantee equal variance. Reuse the baseline and evoked subsets:
- Compute, instead of the mean across samples, the standard deviation (numpy.std or variants)
- calculate the mean for each trace:
    - once for the baseline period
    - once for the remainder of the trial
- Adapt the plotting code for Q2A and plot these distributions of standard deviations in figure Q2B.

In [ ]:
baseline_std = ...
evoked_std = ...

fig_Q2B, ax = plt.subplots(figsize=(8,3), nrows=1, ncols=3) # don't forget proper plot annotation

##
## your code here
##



## Q3 - normalizing the data

So, if there are large differences in mean and/or standard deviation between channels or participants, we can implement some standard scaling. First, let think about why this "standard scaling" (substracting the mean, dividing by std) is important? You will be combining data from different participants for these exercises. They have been possibly recorded at different days, with different gels or electrodes, and thus with different conductivity between participants. We will assume that the recording across videos within one participant remains stable. So, in order to compare and average the recordings from different participants "fairly", we want them to be on more or less the same scale.

We can thus attempt to normalize the signal per participant, by dividing all data per participant by its standard deviation. Let's show the extent of the problem by plotting the participants with the lowest and highest std side by side. Re-create your matrix of std values for the evoked period, but:
- do not subset one participant, but retain all participants (still only selecting 1 channel)
- average the std values for each trace over trials, save in part_std (you will retain one value per participant)
- use np.argmax (and variants) to create two indexes, min_std and max_std that point to the participants with the lowest and highest standard deviations
- calculate and plot the average over trials for these two participants, using different line colors and proper line labeling. 
    - Plot them in the first subplot of a 1x2 subplot
- Observe the scaling difference



In [ ]:
##
## your code here
##

part_std = 
min_std = 
max_std = 

fig_Q3, ax = plt.subplots(figsize=(8,3), ...) # don't forget proper plot annotation
...

Next, we want to use the standard deviation for each channel to normalize (i.e. divide) the data by this value. 
- Re-use the standard deviation per participants calculated above
- Normalize the evoked matrix (per participant) by the participant std (save as evoked_norm)
- plot the **normalized** average over trials for these two participants, using different line colors and proper line labeling. 
    - Plot them in the second subplot of fig Q3
    - The data should now be more or less on the same scale

In [ ]:
##
## your code here
##

evoked_norm = ...

We now have to apply this normalization across all channels. The fairest way is to calculate the grand standard deviation per participant (over all their channels, so the relative scaling between channels remains intact). This will normalize the range of the signal within one participants, so that they will be comparable between participants.

In order to do this: 
- first preallocate a normalized matrix with the same size as comb_data, called comb_data_norm.
- Next, create a loop to go over Participants, and inside the loop:
    - calculate the grand standard deviation per participant 
    - normalization all values by this grand standard deviation
        - Examine the numpy.std documentation to get a single std value across a 3D matrix
- Then, recreate the plot with the trial averages as made for the first subplot of fig_Q2
    - It should now rather resemble the normalized plot as made in the second subplot
- Save this figure as Figure1



In [ ]:
##
## your code here
##

comb_data_norm = ...

for 
    ..
end


fig_Q3, ax = plt.subplots(figsize=(8,3), ...) # 1x1 fig, don't forget proper plot annotation
...

# save Figure Q3C as your Figure 1 for your report
Figure1 = fig_Q3;